In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from models.jump_vasicek import JumpVasicek
from models.vasicek import Vasicek
from evaluators.monte_carlo import MonteCarlo
from copy import copy

In [19]:
class Calibration:
    """Class for calibrating i-rate models"""
    def __init__(self, model_class, n: int, m: int, model_params: dict, optimize_args: tuple):
        """
        model_class, class: Class for creating an interest rate model,
        model_params, dict: Starting parameters for model,
        optimize_args, tuple: Tuple of model parameter names (ex. ("gamma", "mu")) to optimize
        """
        self.n = n
        self.m = m
        self.model_class = model_class
        self.model_params = model_params
        self.optimize_args = optimize_args
    
    def _calculate_error(self, optimize_params: np.array, Ts: np.array, prices: np.array) -> float:
        """
        Computes error (wrt observed prices) for given values of optimize_params (corresponding to optimize_args)
        First value of optmize_params is always r0
        """
        model_params = copy(self.model_params)
        for arg, param in zip(self.optimize_args, optimize_params[1:]):
            model_params[arg] = param
        model = self.model_class(model_params)
        mc = MonteCarlo(model)
        errors = np.empty(len(Ts))
        i = 0
        for price, T in zip(prices, Ts):
            errors[i], _ = mc._simulate_paths_anti(m=self.m, r0=optimize_params[0], n=self.n, T=T) - price
            print(f"Computed for T={T}")
            i += 1
        return np.linalg.norm(errors)
    def calibrate(self, initial_values: tuple, Ts: np.array, prices: np.array):
        """
        initial_values, np.array: same length as optimize_args
        """
        initial_guess = (0.04,)+initial_values # r0 is always guessed as 0
        error_function = lambda optimize_params: self._calculate_error(optimize_params, Ts=Ts, prices=prices) 
        optimal = minimize(error_function, initial_guess, method="Nelder-Mead", callback=lambda xk: print(f"Current param value: {xk}"))
        return optimal

In [20]:
true_model_params = {
    "kappa": 0.5,
    "mu_r": 0.06,
    "sigma": 0.02,
    "mu": 0,
    "gamma": 0.01,
    "h": 10
}

In [21]:
dummy_model_params = {
    "kappa": 0.75,
    "mu_r": 0.07,
    "sigma": 0.05,
    "mu": 0,
    "gamma": 0.01,
    "h": 10
}

In [22]:
vasicek = Vasicek(true_model_params)
mc_vasicek = MonteCarlo(vasicek)

In [23]:
Ts = []
prices = []
for T in range(1, 10):
    Ts.append(T)
    prices.append(vasicek.exact(r0=0.05, T=T))

In [24]:
calibration = Calibration(Vasicek, n=100, m=26, model_params=dummy_model_params, optimize_args=("kappa", "mu_r", "sigma"))

In [25]:
optimal = calibration.calibrate((dummy_model_params["kappa"], dummy_model_params["mu_r"], dummy_model_params["sigma"]), Ts=Ts, prices=prices)

Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T

Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Current param value: [0.04094567 0.79371321 0.0620905  0.05345141]
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Current param value: [0.04112978 0.79193552 0.062148   0.05338365]
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9

Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Current param value: [0.04114864 0.79295292 0.0621187  0.05338462]
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Current param value: [0.04114864 0.79295292 0.0621187  0.05338462]
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Current param value: [0.04114864 0.79295292 0.0621187  0.05338462]
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1


Computed for T=9
Current param value: [0.04114864 0.79291828 0.06212068 0.05338001]
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Current param value: [0.04114864 0.79291828 0.06212068 0.05338001]
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Current param value: [0.04114864 0.79291828 0.06212068 0.05338001]
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Current param val

Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Current param value: [0.04114864 0.79291813 0.06212068 0.05338002]
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Current param value: [0.04114864 0.79291813 0.06212068 0.05338002]
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Current param value: [0.04114864 0.79291813 0.06212068 0.05338002]
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1


...

Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Current param value: [0.04114864 0.79291813 0.06212068 0.05338002]
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Current param value: [0.04114864 0.79291813 0.06212068 0.05338002]
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4

Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Current param value: [0.04114864 0.79291813 0.06212068 0.05338002]
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Current param value: [0.04114864 0.79291813 0.06212068 0.05338002]
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2

Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Current param value: [0.04114864 0.79291813 0.06212068 0.05338002]
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Current param value: [0.04114864 0.79291813 0.06212068 0.05338002]
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2

Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Current param value: [0.04114864 0.79291813 0.06212068 0.05338002]
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Current param value: [0.04114864 0.79291813 0.06212068 0.05338002]
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Current param value: [0.04114864 

Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Current param value: [0.04114864 0.79291813 0.06212068 0.05338002]
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Current param value: [0.04114864 0.79291813 0.06212068 0.05338002]
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2
Computed for T=3
Computed for T=4
Computed for T=5
Computed for T=6
Computed for T=7
Computed for T=8
Computed for T=9
Computed for T=1
Computed for T=2

In [26]:
optimal

 final_simplex: (array([[0.04114864, 0.79291813, 0.06212068, 0.05338002],
       [0.04114864, 0.79291813, 0.06212068, 0.05338002],
       [0.04114864, 0.79291813, 0.06212068, 0.05338002],
       [0.04114864, 0.79291813, 0.06212068, 0.05338002],
       [0.04114864, 0.79291813, 0.06212068, 0.05338002]]), array([0.00444151, 0.00533517, 0.00562237, 0.00574656, 0.00583899]))
           fun: 0.004441507501885862
       message: 'Maximum number of function evaluations has been exceeded.'
          nfev: 800
           nit: 287
        status: 1
       success: False
             x: array([0.04114864, 0.79291813, 0.06212068, 0.05338002])